In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import pandas as pd
PATIENT_LIST = [
    775,  787,  788, 1123, 1169, 1170, 1171, 1172, 1173, 1983, 2110, 2195,
    2955, 2956, 2957, 2958, 2959, 2960, 2961, 2962, 2963, 3081, 3229, 3318, 3432]

In [43]:
from ai_cdss.services.data import DataLoader
loader = DataLoader(PATIENT_LIST)

session = loader.load_session_data()
patient = loader.load_patient_data()
protocol = loader.load_protocol_data()
timeseries = loader.load_timeseries_data()

Database engine created successfully
Data successfully saved to rgs_interaction.csv
Database engine closed
query_timeseries_v3.sql
Database engine created successfully
Data successfully saved to rgs_timeseries.csv
Database engine closed
Function 'fetch_timeseries_data' executed in 9.7659 seconds


In [ ]:
# Patient Protocol
from ai_cdss.services.processing import create_multiindex_df, multiindex
patient_protocol_index = multiindex(patient.index, protocol.PROTOCOL_ID)

In [ ]:
from ai_cdss.services.processing import compute_protocol_similarity
protocol_sim = compute_protocol_similarity(protocol)

In [ ]:
from ai_cdss.services.processing import ClinicalProcessor, ProtocolProcessor, TimeseriesProcessor, SessionProcessor
patient_deficiency = ClinicalProcessor().process(patient)
protocol_mapped = ProtocolProcessor().process(protocol)

timeseries_processed = TimeseriesProcessor().process(timeseries)
session_processed = SessionProcessor().process(session)

from ai_cdss.services.processing import compute_ppf
ppf, contrib = compute_ppf(patient_deficiency, protocol_mapped)

In [44]:
timeseries.to_csv("nest_timeseries_app.csv")

In [13]:
from ai_cdss.services.processing import compute_usage
usage = compute_usage(session, patient_protocol_index)

In [ ]:
from ai_cdss.services.processing import merge_data
from functools import reduce
patient_protocol_table = create_multiindex_df(patient.index, protocol.PROTOCOL_ID)
dfs = [session_processed, timeseries_processed, ppf, contrib, usage]
scoring = reduce(merge_data, [patient_protocol_table] + dfs)

In [ ]:
## INIT THE OUTPUT STRUCTURE

In [ ]:
weights = [1, 1, 1]
scoring['SCORE'] = (
    scoring['ADHERENCE_EWMA'] * weights[0] +
    scoring['PARAMETER_VALUE_EWMA'] * weights[1] +
    scoring['PPF'] * weights[2]
)

In [22]:
from ai_cdss.services.processing import rank_top_n
scoring_ranked = rank_top_n(scoring, 5)

In [25]:
from ai_cdss.services.processing import schedule
prescriptions = schedule(scoring_ranked, days_per_week=7, prescriptions_per_day=5)

Update Prescriptions

In [ ]:
from ai_cdss.services.processing import interchange_mask
prescriptions["INTERCHANGE"] = interchange_mask(prescriptions)

In [ ]:
from ai_cdss.services.processing import substitute_protocol
prescriptions["NEW_PROTOCOL_ID"] = prescriptions.apply(
    substitute_protocol,
    axis=1,
    args=(protocol_sim, usage)
)

In [40]:
prescriptions

,PATIENT_ID,PROTOCOL_ID,ADHERENCE_EWMA,PARAMETER_VALUE_EWMA,PPF,CONTRIB,USAGE,SCORE,DAYS,INTERCHANGE,NEW_PROTOCOL_ID
0,775,222,0.999770,1.000000,0.632038,"[0.022302696143133578, 0.07434232047711194, 0....",10,2.631808,"[5, 6, 7, 3, 1, 2]",False,222
1,775,206,0.853333,1.000000,0.528646,"[0.013972368519644753, 0.13972368519644754, 0....",12,2.381980,"[2, 5, 7, 1, 4]",False,206
2,775,224,1.000000,0.673047,0.632038,"[0.022302696143133578, 0.07434232047711194, 0....",11,2.305085,"[1, 6, 4, 2, 5]",False,224
3,775,208,0.606097,0.994970,0.484665,"[0.02088597029033065, 0.13923980193553767, 0.0...",13,2.085731,"[4, 3, 7, 6]",True,219
4,775,214,0.394473,0.950000,0.617411,"[0.014909525935815035, 0.04969841978605012, 0....",5,1.961884,"[3, 2, 5, 6, 1]",True,223
...,...,...,...,...,...,...,...,...,...,...,...
120,3432,214,NaN,NaN,0.720254,"[0.006777612146718322, 0.0451840809781221, 0.0...",0,NaN,"[6, 7, 1, 4, 2]",False,214
121,3432,223,NaN,NaN,0.727791,"[0.006485284428719559, 0.04323522952479702, 0....",0,NaN,"[3, 4, 7]",False,223
122,3432,208,NaN,NaN,0.606073,"[0.00949440019388549, 0.12659200258513972, 0.0...",0,NaN,"[5, 7, 1, 4]",False,208
123,3432,204,NaN,NaN,0.761949,"[0.006101808684743684, 0.12203617369487356, 0....",0,NaN,"[4, 6, 5, 1, 2, 3]",False,204


In [ ]:
cdss